<h1>Regression Baseline Model</h1>

<h3>Objective</h3><br>
Predict monthly revenue using historical business metrics in order to
estimate expected revenue and understand key revenue drivers.

<h4>1. Imports and Configuration</h4>

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

import matplotlib.pyplot as plt

import joblib

<strong>sklearn (Scikit-learn)</strong> : widely used, free, open-source machine learning library for the Python programming language.<br>
It is used in this project to predict a continuous numerical value which is the <strong>monthly revenue</strong> .

<h4>2. Data Loading</h4>

In [3]:
df=pd.read_csv("../data/raw/small_business_sales.csv")
df.head()

,month,season,marketing_spend,website_visits,conversion_rate,num_customers,avg_order_value,discount_rate,revenue
0,2023-01,Winter,9270,25114,0.0419,1051,65.82,20,51733.52
1,2023-02,Winter,2860,8403,0.0292,245,68.88,10,13249.35
2,2023-03,Spring,7390,22536,0.0428,965,68.94,0,68308.74
3,2023-04,Spring,7191,22896,0.0440,1006,67.55,15,59930.53
4,2023-05,Spring,7734,28443,0.0288,818,57.93,20,41251.89


<h4>3. Feature and Target Definition</h4>

In [4]:
X=df.drop(columns=["revenue"])
y=df["revenue"]

<ul>
<li>X contains all input features</li>
<li>y contains the target (what the model must predict)</li>
</ul>

<h4>4. Train / Validation Split</h4>

In [5]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42 
)

Splitting dataset into:
<ul>
<li>training set (80%) : used to learn patterns</li>
<li>test set (20%) : used to evaluate performance</li>
</ul>

<h4>5. Feature Types</h4>

In [6]:
numerical_features=[
    "marketing_spend",
    "website_visits",
    "conversion_rate",
    "num_customers",
    "avg_order_value",
    "discount_rate"
]
categorical_features=["season"]

Different data types need different preprocessing so this tells the model: 
<ul>
<li>which columns are numerical</li>
<li>which columns are categorical</li>
</ul><br>
→ To allow correct transformations later
<br>

<strong>Note</strong> : The month feature is excluded in the baseline model to avoid improper handling of temporal information.

<h4> 6. Preprocessing Pipelines</h4>

<h5>6.1. Numerical Preprocessing Pipeline</h5>

In [7]:
numeric_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    #replaces missing values(future-proof pipeline)
    #median is robust to outliers
    ("scaler", StandardScaler())
    #centers data around 0
    #makes features comparable in magnitude
])

This prevents from one feature from dominating due to units and keeps preprocessing consistent and reusable.

<h5>6.2. Categorical Preprocessing Pipeline</h5>

In [8]:
categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    #replaces missing categories with the most common value
    ("encode", OneHotEncoder(handle_unknown="ignore"))
    #converts categories into binary columns
    #prevents errors on unseen categories
])

This is important because models can't work with raw text categories.

<h5>6.3. Combine Preprocessing with ColumnTransformer</h5>

In [9]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_pipeline, numerical_features),
        ("cat", categorical_pipeline, categorical_features)
    ]
)
#merges the results into one feature matrix

This is important to keep preprocessing pipelines type-aware and scale cleanly when features grow as well as to prevent manual mistakes.  

<h4> 7. Regression Model Pipeline</h4>

In [10]:
model_pipeline = Pipeline(steps=[
    ("preprocessing", preprocessor),
    ("model", LinearRegression())
])

This :
<ul>
<li> Chains preprocessing and modeling into one object</li>

<li>Ensures preprocessing is fit only on training data applied identically to validation data</li>
</ul><br>
→ Critical for preventing data leakage

<h4> 8. Model Training</h4>

In [11]:
model_pipeline.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessing', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers c

→ Preprocessing is fit on training data<br>
→ Linear regression learns relationships between features and revenue

<h4> 9. Model Evaluation</h4>

In [12]:
y_pred = model_pipeline.predict(X_val)
#applies the trained pipeline to unseen data → produces revenue predictions

The baseline linear regression model provides a first estimate of monthly revenue.

In [13]:
mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

MAE: 6163.96
RMSE: 8297.01


<strong>MAE</strong>:
<ul>
<li>Average absolute error</li>
</ul><br>
<strong>RMSE</strong>:
<ul>
<li>Penalizes large errors more</li>
<li>Sensitive to outliers</li>
</ul><br>

→ On average, the model’s revenue predictions differ from actual values by approximately MAE units.

<strong>This model serves as a reference point for further improvements.</strong>


 ### Final step : **Saving Model for Future Usage**

In [15]:
joblib.dump(model_pipeline, "../models/revenue_prediction_pipeline.pkl")
print("Model saved successfully !")

Model saved successfully !


Saving the pipeline is important to ensure that : 
<ul>
<li>Preprocessing</li>
<li>Encoding</li>
<li>Scaling</li>
<li>Model</li>
</ul>
are applied <strong>exactly</strong> the same way during prediction.